# Creating the First Commitment Transactions

In this section we'll create the First Commitment transaction of a Lightning Channel from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and the message the peers exchange to send and get the needed information from each other to make it happen. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A high level understanding of the Lightning Network. e.g. [Mastering Lightning](https://github.com/bitcoinbook/bitcoinbook) TODO Get the chapter to point it here
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).

- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'

## Commitment Transactions

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md))  details the exact format of on-chain transactions, which both sides need to agree on to ensure signatures are valid. This consists of the funding transaction output script, the commitment transactions, and the HTLC transactions.

This notebook will focus on commitment transactions.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'

Alice send the `open_channel` message to Bob. This message define many channel operational criterias, here we are going to focus on the information that are used in the commitment transaction:
* dust_limit_satoshis
* feerate_per_kw
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

If Bob does not agree with the criterias sent into the `open_channel` message he send a [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice, otherwise he sends the [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, containing some operational criterias from your side, here we are going to focus on the information that are used in the commitment transaction:
* dust_limit_satoshis
* to_self_delay
* funding_pubkey
* revocation_basepoint
* payment_basepoint
* delayed_payment_basepoint
* htlc_basepoint
* first_per_commitment_point

In [1]:
# In your notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 1 - Funding Transactions/Funding-Transaction.ipynb"

Stored 'alice_funding_address' (str)
Alice's pubkey: 037685b7e0a23a1b58540e6af6f60550e2ed6705e858817d92df23170a65e4a4c1
Alice's funding p2wpkh address: bcrt1qk7k7fzan0wfn6tf3y4vylujlvcrs2s5v4d2ngm
Alice's change p2wpkh address: bcrt1qhm34rrrhcntahx9cg8dzsgxeq9xymnkh8rgcrk
2024-12-29T23:23:22.574000Z TestFramework (INFO): PRNG seed is: 2665835548890888750
2024-12-29T23:23:22.575000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_w3tbfybh
txid: ebdb2e3088bdf6d741cb543019cd88122aab038c4d3cb61119541e0a1be247c4, 0
Bob's pubkey: 03e81dfa1a26011178a4c5e8a704e5909015a51558deb011c43d98fd94c63b1223
Alice pubkey is lexicographically lesser than Bob pubkey:
037685b7e0a23a1b58540e6af6f60550e2ed6705e858817d92df23170a65e4a4c1 < 03e81dfa1a26011178a4c5e8a704e5909015a51558deb011c43d98fd94c63b1223
Redeem Script:
5221037685b7e0a23a1b58540e6af6f60550e2ed6705e858817d92df23170a65e4a4c12103e81dfa1a26011178a4c5e8a704e5909015a51558deb011c43d98fd94c63b122352ae
bcrt1qg7zgfcvz9eneugzh9df8

In [2]:
result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

[{'txid': 'dff18cf7a9130e50fc50d25264fa8a9c96bee37f75edfd5874be6efa9456e0dc', 'wtxid': '3acb9b7ee2deae2be728e5e14fd0947acc54b3a1d1f9a04532c9e11415f02e06', 'allowed': True, 'vsize': 153, 'fees': {'base': Decimal('0.00100000'), 'effective-feerate': Decimal('0.00653594'), 'effective-includes': ['3acb9b7ee2deae2be728e5e14fd0947acc54b3a1d1f9a04532c9e11415f02e06']}}]
